데이터 전처리

In [17]:
import pandas as pd

# Load dataset
file_path = 'kma_2022_jeonju_noNULL_noDuplicated.csv'  # 파일 경로를 지정하세요.
data = pd.read_csv(file_path)

# Convert 'TM' column to datetime format
data['TM'] = pd.to_datetime(data['TM'], format='%Y%m%d%H%M')

# Select relevant columns for prediction
selected_columns = ['STN','TM', 'WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 'CA_TOT', 'CA_MID', 'VS', 'TS', 'TE_005', 'TE_01', 'TE_02','TE_03', 'PV', 'PM10']
processed_data = data[selected_columns]

# Sort by time
processed_data = processed_data.sort_values('TM').reset_index(drop=True)

BASE MODEL : ARIMA 모델

In [18]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

arima_data = processed_data[['TM', 'PM10']].set_index('TM')

train_size = int(len(arima_data) * 0.7)
val_size = int(len(arima_data) * 0.15)

train_arima = arima_data.iloc[:train_size]['PM10']
y_val = arima_data.iloc[train_size:train_size + val_size]['PM10']
y_test = arima_data.iloc[train_size + val_size:]['PM10']
arima_order = (5, 1, 0)  # (p, d, q) 값 조정 가능
arima_model = ARIMA(train_arima, order=arima_order)
arima_fit = arima_model.fit()

# 예측
forecast_val = arima_fit.forecast(steps=len(y_val))
forecast_test = arima_fit.forecast(steps=len(y_test))

# 평가
val_rmse = np.sqrt(mean_squared_error(y_val, forecast_val))
val_mae = mean_absolute_error(y_val, forecast_val)
test_rmse = np.sqrt(mean_squared_error(y_test, forecast_test))
test_mae = mean_absolute_error(y_test, forecast_test)

print(f"ARIMA Validation RMSE: {val_rmse}, MAE: {val_mae}")
print(f"ARIMA Test RMSE: {test_rmse}, MAE: {test_mae}")


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


ARIMA Validation RMSE: 19.229282851013295, MAE: 14.400333205730684
ARIMA Test RMSE: 40.08816092897648, MAE: 21.46275262683131


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


MLP model

In [19]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Features (X) and target (y)
X = processed_data.drop(columns=['TM', 'PM10'])
y = processed_data['PM10']

# Split into training, validation, and test sets
train_size = 0.7
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=train_size, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)


# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# MLP 모델 설계
mlp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer
])

mlp_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 학습
history = mlp_model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=50,
    batch_size=32,
    verbose=1
)

# 평가
train_loss = history.history['loss'][-1]  # 마지막 epoch의 train loss
val_loss, val_mae = mlp_model.evaluate(X_val_scaled, y_val, verbose=0)
test_loss, test_mae = mlp_model.evaluate(X_test_scaled, y_test, verbose=0)

print(f"MLP Train Loss (RMSE): {np.sqrt(train_loss)}")
print(f"MLP Validation Loss (RMSE): {np.sqrt(val_loss)}, MAE: {val_mae}")
print(f"MLP Test Loss (RMSE): {np.sqrt(test_loss)}, MAE: {test_mae}")

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 2126.9021 - mae: 22.9321 - val_loss: 261.5714 - val_mae: 12.8292
Epoch 2/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 3115.7236 - mae: 17.7733 - val_loss: 229.6223 - val_mae: 11.8456
Epoch 3/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1666.1584 - mae: 15.3011 - val_loss: 257.2438 - val_mae: 12.2772
Epoch 4/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1675.6478 - mae: 14.1495 - val_loss: 278.3853 - val_mae: 12.7191
Epoch 5/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3710.1223 - mae: 16.1684 - val_loss: 253.3372 - val_mae: 12.1121
Epoch 6/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1050.4406 - mae: 14.2927 - val_loss: 416.5720 - val_mae: 15.3861
Epoch 7/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2830.1946 - mae: 16.5094 - val_loss: 256.3485 - val_mae: 12.0698
Epoch 8/50
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1423.7238 - mae: 14.2730 - val_loss: 284.1098 - val_mae: 12.6238
Epoch 9/50

In [20]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Features (X) and target (y)
X = processed_data.drop(columns=['TM', 'PM10'])
y = processed_data['PM10']

# Split into training, validation, and test sets
train_size = 0.7
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=train_size, shuffle=False)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)


# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# MLP 모델 설계
mlp_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer
])

mlp_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 학습
history = mlp_model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=200,
    batch_size=32,
    verbose=1
)

# 평가
train_loss = history.history['loss'][-1]  # 마지막 epoch의 train loss
val_loss, val_mae = mlp_model.evaluate(X_val_scaled, y_val, verbose=0)
test_loss, test_mae = mlp_model.evaluate(X_test_scaled, y_test, verbose=0)

print(f"MLP Train Loss (RMSE): {np.sqrt(train_loss)}")
print(f"MLP Validation Loss (RMSE): {np.sqrt(val_loss)}, MAE: {val_mae}")
print(f"MLP Test Loss (RMSE): {np.sqrt(test_loss)}, MAE: {test_mae}")

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 2021.1926 - mae: 24.2969 - val_loss: 237.1064 - val_mae: 12.1904
Epoch 2/200
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1348.8230 - mae: 15.9765 - val_loss: 359.1743 - val_mae: 15.1030
Epoch 3/200
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 2598.8486 - mae: 15.9338 - val_loss: 242.7804 - val_mae: 12.0883
Epoch 4/200
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 3901.7090 - mae: 15.9828 - val_loss: 234.1310 - val_mae: 11.6894
Epoch 5/200
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2132.0701 - mae: 14.1598 - val_loss: 266.6599 - val_mae: 12.5368
Epoch 6/200
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 1536.3081 - mae: 15.4650 - val_loss: 263.6764 - val_mae: 12.2557
Epoch 7/200
157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1353.1863 - mae: 13.8054 - val_loss: 292.7709 - val_mae: 12.8521
Epoch 8/200
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2439.2483 - mae: 14.6733 - val_loss: 244.3897 - val_mae: 11.7533
Epoc